In [1]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
import tempfile
from tracing_client import TracingClient

compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")
loggingclient = TracingClient(client)



In [2]:
tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [3]:
application_description = """
generate a hello bot 
""".strip()

In [4]:
typespec = application._make_typespec(application_description)
typespec

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


TypespecOut(reasoning='For a hello bot, I expect simple greeting interactions where the bot responds to users. The main functionality would be:\n1. Greeting users with customizable message\n2. Storing user preferences for greetings\n3. Basic user recognition\n\nThe LLM can extract:\n"Hi, I\'m John" -> setUser({name: "John"})\n"Set my greeting to Good morning!" -> setGreeting({message: "Good morning"})\n"Hello" -> greetUser() // LLM will use stored context about the user\n\nThe interface should be simple but allow for personalization.', typespec_definitions='model UserInfo {\n    name: string;\n    preferredGreeting: string;\n    lastGreeted: utcDateTime;\n}\n\nmodel GreetingMessage {\n    message: string;\n}\n\ninterface HelloBot {\n    @llm_func(1)\n    setUser(user: UserInfo): void;\n\n    @llm_func(1)\n    setGreeting(greeting: GreetingMessage): void;\n\n    @llm_func(2)\n    greetUser(): string;\n}', llm_functions=['setUser', 'setGreeting', 'greetUser'], error_output=None)

In [5]:
typescript_schema = application._make_typescript_schema(typespec.typespec_definitions)
typescript_schema

TypescriptOut(reasoning="Based on the TypeSpec models and interface:\n1. We need to create UserInfo type with string and date fields\n2. GreetingMessage type with a message field\n3. HelloBot interface with three methods marked as LLM functions\n4. For TypeScript, we'll use Date object for utcDateTime\n5. The interface methods should match the TypeSpec definitions", typescript_schema='export interface UserInfo {\n    name: string;\n    preferredGreeting: string;\n    lastGreeted: Date;\n}\n\nexport interface GreetingMessage {\n    message: string;\n}\n\nexport interface HelloBot {\n    setUser(user: UserInfo): void;\n    setGreeting(greeting: GreetingMessage): void;\n    greetUser(): string;\n}', type_names=['UserInfo', 'GreetingMessage', 'HelloBot'], error_output=None)

In [6]:

from policies import drizzle

jinja_env= application.jinja_env
content = jinja_env.from_string(drizzle.PROMPT).render(typespec_definitions=typespec.typespec_definitions)
message = {"role": "user", "content": content}

drizzle_client = TracingClient(client) 
response = drizzle_client.call_anthropic(
            model="anthropic.claude-3-5-sonnet-20241022-v2:0",
            max_tokens=8192,
            messages=[message],
        )
response.content[0].text

'I\'ll analyze the TypeSpec models and interfaces to generate an appropriate Drizzle schema, taking into account the existing tables and following best practices.\n\n<reasoning>\nThe TypeSpec defines:\n1. UserInfo model with name, preferred greeting, and last greeted timestamp\n2. GreetingMessage model with just a message field\n3. HelloBot interface that manages user greetings\n\nWe need to create tables for:\n- UserInfo (extending existing users table)\n- GreetingMessage (for storing greeting templates)\n- We can use the existing messages table for interaction history\n</reasoning>\n\n<drizzle>\nimport { pgTable, text, timestamp, varchar } from "drizzle-orm/pg-core";\nimport { usersTable } from "./common";\nimport { relations } from "drizzle-orm";\n\n// Extend user information\nexport const userInfoTable = pgTable("user_info", {\n  userId: text("user_id")\n    .primaryKey()\n    .references(() => usersTable.id),\n  name: varchar("name", { length: 255 }).notNull(),\n  preferredGreetin

In [7]:
from policies.drizzle import DrizzleTaskNode

reasoning, drizzle_schema = DrizzleTaskNode.parse_output(response.content[0].text)
reasoning, drizzle_schema


('The TypeSpec defines:\n1. UserInfo model with name, preferred greeting, and last greeted timestamp\n2. GreetingMessage model with just a message field\n3. HelloBot interface that manages user greetings\n\nWe need to create tables for:\n- UserInfo (extending existing users table)\n- GreetingMessage (for storing greeting templates)\n- We can use the existing messages table for interaction history',
 'import { pgTable, text, timestamp, varchar } from "drizzle-orm/pg-core";\nimport { usersTable } from "./common";\nimport { relations } from "drizzle-orm";\n\n// Extend user information\nexport const userInfoTable = pgTable("user_info", {\n  userId: text("user_id")\n    .primaryKey()\n    .references(() => usersTable.id),\n  name: varchar("name", { length: 255 }).notNull(),\n  preferredGreeting: text("preferred_greeting").notNull(),\n  lastGreeted: timestamp("last_greeted", { withTimezone: true }).notNull(),\n});\n\n// Store greeting templates\nexport const greetingTemplatesTable = pgTable(

In [8]:

drizzle_compiler = application.compiler
feedback = drizzle_compiler.compile_drizzle(drizzle_schema)
feedback

{'exit_code': 0,
 'stdout': "Reading config file '/app/drizzle.config.ts'\nUsing 'pg' driver for database querying\nsrc:\ncommon\ndb\nhandlers\nhelpers\nlogic\nmain.ts\ntests\n\nsrc/common:\ncrud.ts\nhandler.ts\nindex.ts\nllm.ts\nschema.ts\n\nsrc/db:\nindex.ts\nschema\n\nsrc/db/schema:\napplication.ts\ncommon.ts\n\nsrc/handlers:\ndummy_handler.ts\n\nsrc/helpers:\nreset.ts\n\nsrc/logic:\nindex.ts\nrouter.ts\n\nsrc/tests:\nfeatures\nhandlers\n\nsrc/tests/features:\napplication.feature\n\nsrc/tests/handlers:\ndummy_handler.test.ts\n",
 'stderr': 'TypeError: (0 , import_pg_core.text)(...).primaryKey(...).defaultRandom is not a function\n    at Object.<anonymous> (/app/src/db/schema/application.ts:17:31)\n    at Module._compile (node:internal/modules/cjs/loader:1267:14)\n    at Module._compile (/app/node_modules/drizzle-kit/bin.cjs:13803:30)\n    at Module._extensions..js (node:internal/modules/cjs/loader:1321:10)\n    at Object.newLoader [as .ts] (/app/node_modules/drizzle-kit/bin.cjs:1380

In [9]:
dz = application._make_drizzle(typespec.typespec_definitions)
dz